## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from IPython.display import display

data = pd.read_csv('data.csv')
print(data.info())
display(data.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

- children - 47 человек указали по -(минус)1 ребенку, исправим на 1, еще 76 чел. указали по 20 детей, что вызывает сомнения
- days_employed - отрицательные значения в колонке, пропущенные значения заменим на 0, т к эта колонка не участвует в анализе
- dob_years - 101 чел. указали, что им по 0 лет
- education - разный регистр, приведем к нижнему регистру для поиска дубликатов 
- family_status - разный регистр, приведем к нижнему регистру для поиска дубликатов 
- gender - 1 чел. указал XNA
- total_income - пропущены значения, заменим на значение медианы

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data.isnull().sum()

# отрицательное кол-во детей заменили на положительное число
data['children'] = data['children'].replace(-1, 1)

# пропущенные значения заменим на 0 в таблице 'days_employed'
data['days_employed'] = data['days_employed'].fillna(0)

# переведем все в нижний регистр в таблице 'education'
data['education'] = data['education'].str.lower()

# переведем все в нижний регистр в таблице 'family_status'
data['family_status'] = data['family_status'].str.lower()

# рассчитаем значение медианы и заменим пропущениые значения этим числом
median_total_income = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(median_total_income)

# проверим на кол-во пропущенных значений
data.isnull().sum()



children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Методом isnull() определяем пропущенные значения, а потом для days_employed - значения заменим на 0, тк эта колонка не участвует в анализе,а для total_income - заменим на значение медианы. Причиной пропусков может быть сокрытие информации или потеря части данных.

### Замена типа данных

In [3]:
# заменим вещественные числа в total_income и days_employed на целые числа
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

# проверка
data.info()
#display(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод

используем метод astype(), т к он позволяет перевести данные в необходиммый для нас тип

п.с. возможно необходимо перевести стаж в годы и избавится от отрицательных значений, для простоты восприятия, но эта колонка не участвует в анализе поэтому, наверное, в этом нет необходимости

### Обработка дубликатов

In [4]:
# поиск кол-ва дубликатов
data.duplicated().sum()

# удаление дубликатов
data = data.drop_duplicates().reset_index(drop=True)

# проверка
data.duplicated().sum()



0

### Вывод

Использовался метод duplicated для поиска дубликатов, а для удаления drop_duplicates(). В представленной таблицы нет персональных данных(ФИО, например) по которым можно было бы найти дубликаты при ручном поиске. Возможно, дубликаты появились при обращении в разные отделения банка для офрмления кредита.

### Лемматизация

In [ ]:
m = Mystem()
# находим уникальные значения в столбце 
lem_purpose = data['purpose'].unique()

# проверка и определение значений, по которым в дальнейшем будем категоризировать данные
print(lem_purpose)

# автомобиль, недвижимость/жилье, свадьба, образование

# в функции пропускаем весь столбец через лемматизацию и через цикл присваиваем новые значения в новом столбце
def lemmas_purpose(purpose):
    lemma = m.lemmatize(purpose)
    for value in lemma:
        if 'автомоб' in value:
            return 'автомобиль'
        elif 'образов' in value:
            return 'образование'
        elif 'свадьб' in value:
            return 'свадьба'
        elif 'жил' in value or 'недвиж' in value:
            return 'недвижимость'

    
data['lemmas_purpose'] = data['purpose'].apply(lemmas_purpose)

# проверка
data['lemmas_purpose'].value_counts()

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


### Вывод

сначала нашли уникальные значения в солбце 'purpose', распечатали и определелили, что основными категориями могут быть: автомобиль, недвижимость/жилье, свадьба и образование. Далее в функции пропускаем весь столбец через лемматизацию и через цикл находим соответствующие значения и присваиваем новые значения в новом столбце.


### Категоризация данных

In [7]:
# категоризация по уровню дохода
def salary(payday):
    if payday <= 100000:
        return 'низкий'
    elif 100000 < payday <= 140000:
        return 'средний'
    elif 140000 < payday <= 180000:
        return 'высокий'
    elif  payday > 180000:
        return 'очень высокий'
    
        
data['salary'] = data['total_income'].apply(salary)

#проверка
display(data['salary'].value_counts())


очень высокий    6422
высокий          5927
средний          4642
низкий           4463
Name: salary, dtype: int64


### Вывод

создали категоризацию по уровню дохода, а также выше была создана по целям кредита.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
# кол-во семей с детьми
parent_child = data[data['children'] >= 1]['children'].count()

# кол-во семей без детей
parent_not_child = data[data['children'] == 0]['children'].count()

# кол-во семей с детьми, которые не имеют задолжности
parent_child_not_debt = data[(data['debt'] == 0) & (data['children'] >= 1)]['children'].count()

# кол-во семей без детей, которые не имеют задолжности
parent_not_child_not_debt = data[(data['debt'] == 0) & (data['children'] == 0)]['children'].count()

#найдем процентное соотношение 
interest_child = parent_child_not_debt / parent_child
interest_not_child = parent_not_child_not_debt / parent_not_child

print('Процент семей с детьми, которые не имеют задолжностей по кредиту {:.1%}'.format(interest_child))
print('Процент семей без детей, которые не имеют задолжностей по кредиту {:.1%}'.format(interest_not_child))
print()

#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_child = data.pivot_table(index=['children'], columns='debt', values='education', aggfunc='count')
# обработаем полученные результаты
pivot_child[0] = pivot_child[0].astype('int')
pivot_child[1] = pivot_child[1].fillna(0)
pivot_child[1] = pivot_child[1].astype('int')
# посчитаем долю людей с разным кол-вом детей, которые не имеют задолжности
pivot_child['interest_child'] = pivot_child[0] / (pivot_child[0] + pivot_child[1])
# представим в процентах
pivot_child['percent'] = pivot_child['interest_child'] * 100
# поменяем название столбцов
#pivot_child.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 
pivot_child.rename(index = {0 : "Нет детей", 1 : "1 ребенок", 2 : "2 ребенка", 3 :"3 ребенка", 
                            4 : "4 ребенка", 5 :"5 детей", 20 : "20 детей"},
                   columns = { 0 : "Нет задолжности", 1 :"Есть задолжность", "interest_child": "Доля", 
                              "percent": "В процентах"}, inplace = True)
display(pivot_child)


Процент семей с детьми, которые не имеют задолжностей по кредиту 90.8%
Процент семей без детей, которые не имеют задолжностей по кредиту 92.5%



debt,Нет задолжности,Есть задолжность,Доля,В процентах
children,,,,
Нет детей,13028,1063,0.924562,92.456178
1 ребенок,4410,445,0.908342,90.834192
2 ребенка,1858,194,0.905458,90.545809
3 ребенка,303,27,0.918182,91.818182
4 ребенка,37,4,0.902439,90.243902
5 детей,9,0,1.000000,100.000000
20 детей,68,8,0.894737,89.473684


### Вывод

В семьях без детей процент вовремя выплаченных кредитов больше на 1,7%, чем в семьях имеющих детей. Возможно, это связано с достаточно дорогим содержанием детей, что приводит к чуть большему кол-ву задолжностей по кредиту, чем в семьях без детей.
В целом в семьях с разным кол-вом детей процент вовремя выплаченных кредитов ниже, чем в семьях без детей(исключение семьи, где 5 детей).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_family_status = data.pivot_table(index=['family_status'], columns='debt', values = 'gender', aggfunc='count')

# посчитаем долю людей с разным семейным положением, которые не имеют задолжности
pivot_family_status['family_stat'] = pivot_family_status[0] / (pivot_family_status[0] + pivot_family_status[1])

# представим в процентах
pivot_family_status['percent'] = pivot_family_status['family_stat'] * 100

# поменяем название столбцов
# pivot_family_status.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 
pivot_family_status.rename(columns = {0 : "Нет задолжности", 1 :"Есть задолжность", "family_stat": "Доля", 
                              "percent": "В процентах"}, inplace = True)


display(pivot_family_status)


debt,Нет задолжности,Есть задолжность,Доля,В процентах
family_status,,,,
в разводе,1110,85,0.928870,92.887029
вдовец / вдова,896,63,0.934307,93.430657
гражданский брак,3763,388,0.906529,90.652855
женат / замужем,11408,931,0.924548,92.454818
не женат / не замужем,2536,274,0.902491,90.249110


### Вывод

люди побывавшие в браке(не гражданском) или находящиеся сейчас в браке, имеют задолжности реже на 2-3%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_salary = data.pivot_table(index=['salary'], columns='debt', values = 'gender', aggfunc='count')

# посчитаем долю людей с разными доходами, которые не имеют задолжности
pivot_salary['salary_level'] = pivot_salary[0] / (pivot_salary[0] + pivot_salary[1])

# представим в процентах
pivot_salary['percent'] = pivot_salary['salary_level'] * 100

# поменяем название столбцов
# pivot_salary.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 
pivot_salary.rename(columns = {0 : "Нет задолжности", 1 :"Есть задолжность", "salary_level": "Доля", 
                              "percent": "В процентах"}, inplace = True)


display(pivot_salary)

debt,Нет задолжности,Есть задолжность,Доля,В процентах
salary,,,,
высокий,5410,517,0.912772,91.277206
низкий,4109,354,0.920681,92.068116
очень высокий,5944,478,0.925568,92.556836
средний,4250,392,0.915554,91.555364


### Вывод

можно сказать, что уровень дохода, практически не влияет на задолжность по кредиту, все результаты с разностью чуть более 1%.

- Как разные цели кредита влияют на его возврат в срок?

In [1]:
#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_lemmas_purpose = data.pivot_table(index=['lemmas_purpose'], columns='debt', values = 'gender', aggfunc='count')

# посчитаем долю людей с разными целями кредита, которые не имеют задолжности
pivot_lemmas_purpose['purpos'] = pivot_lemmas_purpose[0] / (pivot_lemmas_purpose[0] + pivot_lemmas_purpose[1])

# представим в процентах
pivot_lemmas_purpose['percent'] = pivot_lemmas_purpose['purpos'] * 100

# поменяем название столбцов
# pivot_lemmas_purpose.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 
pivot_lemmas_purpose.rename(columns = {0 : "Нет задолжности", 1 :"Есть задолжность", "purpos": "Доля", 
                              "percent": "В процентах"}, inplace = True)


display(pivot_lemmas_purpose)

NameError: name 'data' is not defined

### Вывод

меньший процент c задолжностями по кредитам для недвижимости и свадьбы можно объяснить тем, что чаще всего их берут люди(семья), в которой несколько источников доходов (работающие муж и жена, например). А автомобили и образование чаще берут люди с одним источником доходов. 

### Шаг 4. Общий вывод

- В семьях без детей процент вовремя выплаченных кредитов больше на 1,7%, чем в семьях имеющих детей
- люди побывавшие в браке(не гражданском) или находящиеся сейчас в браке, имеют задолжности реже на 2-3%
- уровень дохода, практически не влияет на задолжность по кредиту, все результаты с разностью меньше 1%
- процент c задолжностями по кредитам для недвижимости и свадьбы менеьше на 1-2%, чем на автомобили и образование

Процент отданных вовремя кредитов находится в диапозоне от 90% до 93,4%(не берем 100%, где 5 детей в семье - маленькая выборка и уберем 89,4% где 20 детей, есть основания думать, что это ошибочные данные). Наиболее часто отдающие кредит люди - это люди без детей, бывшие в законном браке или состоящие на данный момент в браке и приобретающие недвежимость. При этом уровень дохода почти не влияет на задолжность.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.